## 20021016 python勉強会 tkinter2
・クラスの継承による画面作成   
・画面遷移  
・プログレスバーの更新（並列処理）  
を実装します。

In [30]:
#各種モジュールのインポート
import time
import threading
import pandas as pd
import numpy as np
import japanize_matplotlib
import tkinter as tk
import tkinter.ttk as ttk
py3 = True

from tkinter import filedialog,messagebox
from PIL import Image,ImageTk


### 前回の簡単な復習

In [11]:
#ここのセルを実行して、画面が表示されることを確認してください。
#ボタンが３つあるだけの非常にシンプルなGUIが表示されます。

def support_init(top, gui, *args, **kwargs):
    """画面の初期設定する関数"""
    global w, top_level, root
    w = gui
    top_level = top
    root = top

def vp_start_gui():
    '''画面を表示するための関数（Pageで自動作成される関数）'''
    global val, w, root
    root = tk.Tk()#画面を定義する。
    top = tkiner_demo (root)
    support_init(root, top)
    root.mainloop()

class tkiner_demo:
    def __init__(self, top=None):
        '''This class configures and populates the toplevel window.
           top is the toplevel containing window.'''
        _bgcolor = '#d9d9d9'  # X11 color: 'gray85'
        _fgcolor = '#000000'  # X11 color: 'black'
        _compcolor = '#d9d9d9' # X11 color: 'gray85'
        _ana1color = '#d9d9d9' # X11 color: 'gray85'
        _ana2color = '#ececec' # Closest X11 color: 'gray92'
        font10 = "-family {Yu Gothic UI} -size 12"
        font11 = "-family {ＭＳ ゴシック} -size 12"
        font9 = "-family {Yu Gothic UI} -size 14"
        self.style = ttk.Style()
        if sys.platform == "win32":
            self.style.theme_use('winnative')
        self.style.configure('.',background=_bgcolor)
        self.style.configure('.',foreground=_fgcolor)
        self.style.configure('.',font="TkDefaultFont")
        self.style.map('.',background=
            [('selected', _compcolor), ('active',_ana2color)])

        ###############################################################
        #ここから確認してください。
        top.geometry("400x400+616+22")#画面サイズの設定
        top.minsize(120, 1)#画面の縮小できる最小サイズ
        top.maxsize(3204, 1061)#画面を拡大できる最大サイズ
        top.resizable(1, 1)
        top.title("テスト画面")#画面のタイトル
        top.configure(background="#d9d9d9")#背景色

        self.Button_changepage = tk.Button(top)#ボタンの設定
        self.Button_changepage.place(relx=0.65, rely=0.85, height=44, width=117)#位置
        self.Button_changepage.configure(activebackground="#ececec")
        self.Button_changepage.configure(activeforeground="#000000")
        self.Button_changepage.configure(background="#d9d9d9")
        self.Button_changepage.configure(disabledforeground="#a3a3a3")
        self.Button_changepage.configure(foreground="#000000")
        self.Button_changepage.configure(font="-family {Yu Gothic UI} -size 14")#文字フォント
        self.Button_changepage.configure(highlightbackground="#d9d9d9")
        self.Button_changepage.configure(highlightcolor="black")
        self.Button_changepage.configure(pady="0")
        self.Button_changepage.configure(text="推論画面へ")#文字

        self.Button_start = tk.Button(top)#ボタンの設定
        self.Button_start.place(relx=0.35, rely=0.85, height=44, width=117)#位置
        self.Button_start.configure(activebackground="#ececec")
        self.Button_start.configure(activeforeground="#000000")
        self.Button_start.configure(background="#d9d9d9")
        self.Button_start.configure(disabledforeground="#a3a3a3")
        self.Button_start.configure(foreground="#000000")
        self.Button_start.configure(font="-family {Yu Gothic UI} -size 14")#文字フォント
        self.Button_start.configure(highlightbackground="#d9d9d9")
        self.Button_start.configure(highlightcolor="black")
        self.Button_start.configure(pady="0")
        self.Button_start.configure(text="学習開始")#文字

        self.Button_stop = tk.Button(top)#ボタンの設定
        self.Button_stop.place(relx=0.05, rely=0.85, height=44, width=117)#位置
        self.Button_stop.configure(activebackground="#ececec")
        self.Button_stop.configure(activeforeground="#000000")
        self.Button_stop.configure(background="#d9d9d9")
        self.Button_stop.configure(disabledforeground="#a3a3a3")
        self.Button_stop.configure(foreground="#000000")
        self.Button_stop.configure(font="-family {Yu Gothic UI} -size 14")#文字フォント
        self.Button_stop.configure(highlightbackground="#d9d9d9")
        self.Button_stop.configure(highlightcolor="black")
        self.Button_stop.configure(pady="0")
        self.Button_stop.configure(text="学習中止")#文字

if __name__ == '__main__':
    vp_start_gui()


### 上の画面を改良していき、立派なGUIを作成します。

###  問題１：画面の共通機能をまとめるBaseというクラスを作り、tkinter_demoに継承してください。
今回ではボタンが2つありますが、「位置」と「文字」以外のコードは共通しています。このようなコードは毎回書くと煩雑になり分かりにくくなるので、Baseクラスに「button_setting」という関数を作り、まとめてしまいましょう。  

↓位置と文字の情報は変数として可変にしておく必要がある。  
self.Button_start.place(relx=0.3, rely=0.85, height=44, width=117)#位置  
self.Button_start.configure(text="学習開始")#文字  

↓これらのコードは共通項目なので、Baseに関数を作ってまとめてしまいましょう。  
self.Button_start.configure(activebackground="#ececec")   
self.Button_start.configure(activeforeground="#000000")  
self.Button_start.configure(background="#d9d9d9")  
self.Button_start.configure(disabledforeground="#a3a3a3")  
self.Button_start.configure(foreground="#000000")  
self.Button_start.configure(font="-family {Yu Gothic UI} -size 14")#文字フォント  
self.Button_start.configure(highlightbackground="#d9d9d9")  
self.Button_start.configure(highlightcolor="black")  
self.Button_start.configure(pady="0")  


In [3]:
#以下の部分を穴埋めしてください。
class Base:
    """画面の共通要素をまとめておくクラス"""
    def __init__(self):
        self.base = 0#特に意味なし。

    def button_setting(self,model,relx,rely,height,width,text):
        """
        ボタンのパラメータ設定
        Parameters
        －－－－－－－－－－－－－－
        model：定義したボタン（self.Button_to_predictなど）
        relx：画面の大きさに対するx座標（０～１）
        rely：画面の大きさに対するy座標（０～１）
        height：ボタンの高さ
        width：ボタンの幅
        text：ボタンに表示する文字
        """
        #########################################################
        #ここからコードを記述してください。
        #tkinter_demoのbuttonの書き方をコピーしてきて、self.Button_to_predict→modelなど
        #関数の引数に変更してください。
        #また、relx,rely,height,width,textは任意に変えられるようにしてください。
        model.place(relx=relx, rely=rely, height=height, width=width)
        model.configure(activebackground="#ececec")
        model.configure(activeforeground="#000000")
        model.configure(background="#d9d9d9")
        model.configure(disabledforeground="#a3a3a3")
        model.configure(foreground="#000000")
        model.configure(font="-family {Yu Gothic UI} -size 14")
        model.configure(highlightbackground="#d9d9d9")
        model.configure(highlightcolor="black")
        model.configure(pady="0")
        model.configure(text=text)

        #########################################################



In [12]:
#以下を実行して、復習内容と同じ画面が出るようにしてください。

class tkiner_demo(Base):#ここでBaseクラスを継承しています！
    def __init__(self, top=None):
        '''This class configures and populates the toplevel window.
           top is the toplevel containing window.'''
        _bgcolor = '#d9d9d9'  # X11 color: 'gray85'
        _fgcolor = '#000000'  # X11 color: 'black'
        _compcolor = '#d9d9d9' # X11 color: 'gray85'
        _ana1color = '#d9d9d9' # X11 color: 'gray85'
        _ana2color = '#ececec' # Closest X11 color: 'gray92'
        font10 = "-family {Yu Gothic UI} -size 12"
        font11 = "-family {ＭＳ ゴシック} -size 12"
        font9 = "-family {Yu Gothic UI} -size 14"
        self.style = ttk.Style()
        if sys.platform == "win32":
            self.style.theme_use('winnative')
        self.style.configure('.',background=_bgcolor)
        self.style.configure('.',foreground=_fgcolor)
        self.style.configure('.',font="TkDefaultFont")
        self.style.map('.',background=
            [('selected', _compcolor), ('active',_ana2color)])

        ###############################################################
        #ここから確認してください。
        #だいぶスッキリしたことがわかると思います。
        top.geometry("400x400+616+22")#画面サイズの設定
        top.minsize(120, 1)#画面の縮小できる最小サイズ
        top.maxsize(3204, 1061)#画面を拡大できる最大サイズ
        top.resizable(1, 1)
        top.title("学習画面")#画面のタイトル
        top.configure(background="#d9d9d9")#背景色

        self.Button_changepage = tk.Button(top)#ボタンの設定
        self.button_setting(self.Button_changepage,relx=0.65, rely=0.85, 
                            height=44, width=117,text="推論画面へ")#Baseにあるbutton_setting関数を継承している。

        self.Button_start = tk.Button(top)#ボタンの設定
        self.button_setting(self.Button_start,relx=0.35, rely=0.85, 
                            height=44, width=117,text="学習開始")#Baseにあるbutton_setting関数を継承している。

        self.Button_stop = tk.Button(top)#ボタンの設定
        self.button_setting(self.Button_stop,relx=0.05, rely=0.85, 
                            height=44, width=117,text="学習中止")#Baseにあるbutton_setting関数を継承している。
        

if __name__ == '__main__':
    vp_start_gui()

### 問題２：tkinter_demo2という別画面クラスを作成して表示してください。
tkiner_deomo2の画面構成はtkiner_demoとほぼ同じで、ボタンの文字が「推論画面へ」→「学習画面へ」「学習開始」→「推論開始」と変更されるだけです。  
そのためtkiner_demoクラスをtkiner_demo2に継承して、self.button_settingを上書き（オーバーライド）することで、簡単に画面を作成できます。


In [14]:
#tkiner_demo2用の関数作成
def support_init2(top, gui, *args, **kwargs):
    """画面の初期設定する関数"""
    global w, top_level, root
    w = gui
    top_level = top
    root = top

#tkiner_demo2用の関数作成
def vp_start_gui2():
    '''画面を表示するための関数（Pageで自動作成される関数）'''
    global val, w, root
    root = tk.Tk()#画面を定義する。
    top = tkiner_demo2 (root)
    support_init2(root, top)
    root.mainloop()


class tkiner_demo2(tkiner_demo):#tkiner_demoの継承
    def __init__(self,top=None):
        super().__init__(top)#tkiner_demoの__init__を継承する。（ここ重要！）

        ############################################################
        #ここから以下の点を上書きしてください。
        #・画面titleを「推論画面」に変更してください
        #・button_settingを使って、ボタン2つの文字を上書きしてください。
        #  推論画面へ→学習画面へ、学習開始→推論開始
        top.title("推論画面")#画面のタイトル
        #self.Button_changepage = tk.Button(top)#ボタンの設定
        self.button_setting(self.Button_changepage,relx=0.65, rely=0.85, 
                            height=44, width=117,text="学習画面へ")
        
        #self.Button_start = tk.Button(top)#ボタンの設定
        self.button_setting(self.Button_start,relx=0.35, rely=0.85, 
                            height=44, width=117,text="推論開始")
        
        self.button_setting(self.Button_stop,relx=0.05, rely=0.85, 
                            height=44, width=117,text="推論中止")

if __name__ == '__main__':
    vp_start_gui2()

### 問題３：tkiner_demoとtkiner_demo2を使って、画面遷移を実現します。遷移の考え方は非常にシンプルで
### Step1：もともとあった画面を消す。
### Step2：遷移先の画面を表示する
### です。画面遷移はtkiner_demoとtkiner_demo2に共通している要素なので、BaseクラスにchangePageという関数を作ってみましょう。

In [6]:
#以下の部分を穴埋めしてください。
class Base:
    """画面の共通要素をまとめておくクラス"""
    def __init__(self):
        self.base = 0#特に意味なし。

    def button_setting(self,model,relx,rely,height,width,text):
        """
        ボタンのパラメータ設定
        Parameters
        －－－－－－－－－－－－－－
        model：定義したボタン（self.Button_to_predictなど）
        relx：画面の大きさに対するx座標（０～１）
        rely：画面の大きさに対するy座標（０～１）
        height：ボタンの高さ
        width：ボタンの幅
        text：ボタンに表示する文字
        """
        #########################################################
        #ここからコードを記述してください。
        #tkinter_demoのbuttonの書き方をコピーしてきて、self.Button_to_predict→modelなど
        #関数の引数に変更してください。
        model.place(relx=relx, rely=rely, height=height, width=width)
        model.configure(activebackground="#ececec")
        model.configure(activeforeground="#000000")
        model.configure(background="#d9d9d9")
        model.configure(disabledforeground="#a3a3a3")
        model.configure(foreground="#000000")
        model.configure(font="-family {Yu Gothic UI} -size 14")
        model.configure(highlightbackground="#d9d9d9")
        model.configure(highlightcolor="black")
        model.configure(pady="0")
        model.configure(text=text)

        #########################################################

    def changePage(self,start_gui,top):
        """
        画面遷移の関数
        start_gui：vp_start_guiとvp_start_gui2が入ります。（changePage関数内で実行します）
        top：現在、実行中の画面。tkinerにはtop.destroy()で今ある画面を消すことができます。
        """
        ##########################################################
        #ここからコードを記述してください。2行で完結します。
        top.destroy()
        start_gui()



In [22]:
#次にtkiner_demoとtkiner_demo2でchangePageをbutton_changepageに適用します。
#ボタンが押されたときに、実行する形はtk.Button(top,command = lambda : function())という書き方です。

class tkiner_demo(Base):#ここでBaseクラスを継承しています！
    def __init__(self, top=None):
        '''This class configures and populates the toplevel window.
           top is the toplevel containing window.'''
        _bgcolor = '#d9d9d9'  # X11 color: 'gray85'
        _fgcolor = '#000000'  # X11 color: 'black'
        _compcolor = '#d9d9d9' # X11 color: 'gray85'
        _ana1color = '#d9d9d9' # X11 color: 'gray85'
        _ana2color = '#ececec' # Closest X11 color: 'gray92'
        font10 = "-family {Yu Gothic UI} -size 12"
        font11 = "-family {ＭＳ ゴシック} -size 12"
        font9 = "-family {Yu Gothic UI} -size 14"
        self.style = ttk.Style()
        if sys.platform == "win32":
            self.style.theme_use('winnative')
        self.style.configure('.',background=_bgcolor)
        self.style.configure('.',foreground=_fgcolor)
        self.style.configure('.',font="TkDefaultFont")
        self.style.map('.',background=
            [('selected', _compcolor), ('active',_ana2color)])

        #ここから確認してください。
        top.geometry("400x400+616+22")#画面サイズの設定
        top.minsize(120, 1)#画面の縮小できる最小サイズ
        top.maxsize(3204, 1061)#画面を拡大できる最大サイズ
        top.resizable(1, 1)
        top.title("学習画面")#画面のタイトル
        top.configure(background="#d9d9d9")#背景色

        ###################################################################
        #self.Button_changepageを修正してください。
        self.Button_changepage = tk.Button(top,command = lambda : self.changePage(vp_start_gui2,top))#ボタンの設定
        self.button_setting(self.Button_changepage,relx=0.65, rely=0.85, 
                            height=44, width=117,text="推論画面へ")#Baseにあるbutton_setting関数を継承している。

        self.Button_start = tk.Button(top)#ボタンの設定
        self.button_setting(self.Button_start,relx=0.35, rely=0.85, 
                            height=44, width=117,text="学習開始")#Baseにあるbutton_setting関数を継承している。

        self.Button_stop = tk.Button(top)#ボタンの設定
        self.button_setting(self.Button_stop,relx=0.05, rely=0.85, 
                            height=44, width=117,text="学習中止")#Baseにあるbutton_setting関数を継承している。

class tkiner_demo2(tkiner_demo):#tkiner_demoの継承
    def __init__(self,top=None):
        super().__init__(top)#tkiner_demoの__init__を継承する。（ここ重要！）

        ############################################################
        #ここから以下の点を上書きしてください。
        #self.Button_changepageを修正してください。
        top.title("推論画面")#画面のタイトル
        self.Button_changepage = tk.Button(top,command = lambda : self.changePage(vp_start_gui,top))#ボタンの設定
        self.button_setting(self.Button_changepage,relx=0.65, rely=0.85, 
                            height=44, width=117,text="学習画面へ")
        
        #self.Button_start = tk.Button(top)#ボタンの設定
        self.button_setting(self.Button_start,relx=0.35, rely=0.85, 
                            height=44, width=117,text="推論開始")

        self.button_setting(self.Button_stop,relx=0.05, rely=0.85, 
                            height=44, width=117,text="推論中止")#Baseにあるbutton_setting関数を継承している。

#tkiner_demoから実行します。
if __name__ == '__main__':
    vp_start_gui()

### 問題４：最後に並列処理によるプログレスバーの更新と中断を実装してみます。（これを1週間で理解できたら相当すごいです。難易度高めです。）
今回は、プログレスバーの更新にあたり必要な関数を4つ定義します。  
・プログレスバーを更新する関数（progress_step）  
・プログレスバーの更新を中止する関数（progress_stop）  
・プログレスバーを更新するためのループ処理関数（start_progress_bar）  
・プログレスバーの更新を並列処理させる関数。(run_threading)  

#### 今回は、ループ処理関数と並列処理関数を実装してもらいます。  

Q：なぜプログレスバー更新を並列処理にするのか？  
並列処理で出さない場合、画面上のボタンを押せなくなるから。（中止ボタンを押せず、プログラム処理が終わるのを待つしかない。)    
AIモデル学習など時間がかかる処理を走らせるときには、中止したい場面もあるため、必須の処理である。

In [45]:

#tkiner_demoにプログレスバーを定義します。
class tkiner_demo(Base):#ここでBaseクラスを継承しています！
    def __init__(self, top=None):
        '''This class configures and populates the toplevel window.
           top is the toplevel containing window.'''
        _bgcolor = '#d9d9d9'  # X11 color: 'gray85'
        _fgcolor = '#000000'  # X11 color: 'black'
        _compcolor = '#d9d9d9' # X11 color: 'gray85'
        _ana1color = '#d9d9d9' # X11 color: 'gray85'
        _ana2color = '#ececec' # Closest X11 color: 'gray92'
        font10 = "-family {Yu Gothic UI} -size 12"
        font11 = "-family {ＭＳ ゴシック} -size 12"
        font9 = "-family {Yu Gothic UI} -size 14"
        self.style = ttk.Style()
        if sys.platform == "win32":
            self.style.theme_use('winnative')
        self.style.configure('.',background=_bgcolor)
        self.style.configure('.',foreground=_fgcolor)
        self.style.configure('.',font="TkDefaultFont")
        self.style.map('.',background=
            [('selected', _compcolor), ('active',_ana2color)])

        #ここから確認してください。
        top.geometry("400x400+616+22")#画面サイズの設定
        top.minsize(120, 1)#画面の縮小できる最小サイズ
        top.maxsize(3204, 1061)#画面を拡大できる最大サイズ
        top.resizable(1, 1)
        top.title("学習画面")#画面のタイトル
        top.configure(background="#d9d9d9")#背景色

        
        self.Button_changepage = tk.Button(top,command = lambda : self.changePage(vp_start_gui2,top))#ボタンの設定
        self.button_setting(self.Button_changepage,relx=0.65, rely=0.85, 
                            height=44, width=117,text="推論画面へ")#Baseにあるbutton_setting関数を継承している。

        self.Button_start = tk.Button(top,command = lambda : self.run_thread())#スレッド処理の開始
        self.button_setting(self.Button_start,relx=0.35, rely=0.85, 
                            height=44, width=117,text="学習開始")#Baseにあるbutton_setting関数を継承している。

        self.Button_stop = tk.Button(top,command = lambda : self.progress_stop())#処理の中止
        self.button_setting(self.Button_stop,relx=0.05, rely=0.85, 
                            height=44, width=117,text="学習中止")#Baseにあるbutton_setting関数を継承している。

        ###################################################################
        #進捗状況というラベルの作成
        self.Label_progress = tk.Label(top)
        self.Label_progress.place(relx=0.1, rely=0.4, height=40, width=75)
        self.Label_progress.configure(background="#d9d9d9")
        self.Label_progress.configure(font="-family {Yu Gothic UI} -size 14")
        self.Label_progress.configure(text="進捗状況")

        #################################################################
        #プログレスバーの定義
        self.Progressbar = ttk.Progressbar(top,mode='determinate')
        self.Progressbar.place(relx=0.1, rely=0.5, relwidth=0.8
                , relheight=0.0, height=40)
        self.Progressbar.configure(maximum=101,length="488")


    def progress_step(self,step):
        """プログレスバーをステップで更新する関数"""
        self.Progressbar.configure(value=step)
        self.Progressbar.update()
    
    def progress_stop(self):
        """プログレスバーの停止"""
        self.stop_flag = True#中止フラグ
        self.Progressbar.stop()#プログレスバー進捗の中止
        self.Progressbar.configure(value=0)#プログレスバーを0に設定する。

    def start_progress_bar(self):
        """プログレスバーを進捗させる関数"""
        self.stop_flag = False#中止フラグ
        for num in range(101):#101まで繰り返す。
            ########################################################
            #以下を埋めてください。
            #考え方は、「もし中止フラグが立った場合に、この関数処理を抜け出す。」
            #その時に、messagebox.showinfoを使ってユーザーに学習が中止したことを伝える。
            if self.stop_flag == True:
                messagebox.showinfo("中止","学習を中止しました。")
                del self.threading_learn #並列の破棄
                return 
            ########################################################

            self.progress_step(num)#プログレスバーを更新し続ける。
            time.sleep(0.5)
        messagebox.showinfo("学習完了","学習が完了しました。")

    def run_thread(self):
        """並列処理の関数"""
        ###########################################################
        #以下を埋めてください。
        #threading.Thead()、「定義した並列処理」.start()の2つを使って2行で書けます。
        
        self.threading_learn = threading.Thread(target = self.start_progress_bar)
        self.threading_learn.start()


class tkiner_demo2(tkiner_demo):#tkiner_demoの継承
    def __init__(self,top=None):
        super().__init__(top)#tkiner_demoの__init__を継承する。（ここ重要！）

        ############################################################
        #ここから以下の点を上書きしてください。
        #self.Button_changepageを修正してください。
        top.title("推論画面")#画面のタイトル
        self.Button_changepage = tk.Button(top,command = lambda : self.changePage(vp_start_gui,top))#ボタンの設定
        self.button_setting(self.Button_changepage,relx=0.65, rely=0.85, 
                            height=44, width=117,text="学習画面へ")
        
        #self.Button_start = tk.Button(top)#ボタンの設定
        self.button_setting(self.Button_start,relx=0.35, rely=0.85, 
                            height=44, width=117,text="推論開始")

        self.button_setting(self.Button_stop,relx=0.05, rely=0.85, 
                            height=44, width=117,text="推論中止")#Baseにあるbutton_setting関数を継承している。


if __name__ == '__main__':
    vp_start_gui()